> ### Primary Constructor feature 
>    Version *C# 12.0*  and available with .NET8.0 SDK

> **Primary Constructor feature**:  C# 12 introduces primary constructors,a concise syntax to declare constructors whose parameters are available anywhere in the body of the type.  
>   We can add parameters to a struct or class declaration to create a primary constructor. Primary constructor parameters are in scope throughout the class definition.
>   It's important to view primary constructor parameters as parameters even though they are in scope throughout the class definition. 
>   Several rules clarify that they're parameters:
>
>    - Primary constructor parameters may not be stored if they aren't needed.
>    - Primary constructor parameters aren't members of the class. For example, a primary constructor parameter named param can't be accessed as this.param.
>    - Primary constructor parameters can be assigned to.
>    - Primary constructor parameters don't become properties, except in record types.
>
>  These rules are the same as parameters to any method, including other constructor declarations.
> 
> The most common uses for a primary constructor parameter are:
>   -   As an argument to a base() constructor invocation.
>   -   To initialize a member field or property.
>   - Referencing the constructor parameter in an instance member.
>
> Here is the example of Primary Constructor feature

In [ ]:
public class Point(int x, int y) {
    public int X { get; } = x;
    public int Y { get; } = y;
}

Point point = new Point(3, 4);
Console.WriteLine(point.X); // Output: 3
Console.WriteLine(point.Y); // Output: 4


In [ ]:
public readonly struct Distance(double dx, double dy)
{
    public readonly double Magnitude { get; } = Math.Sqrt(dx * dx + dy * dy);
    public readonly double Direction { get; } = Math.Atan2(dy, dx);
}

>  The preceding struct is as though we had written the following code:

In [ ]:
public readonly struct Distance
{
    public readonly double Magnitude { get; }

    public readonly double Direction { get; }

    public Distance(double dx, double dy)
    {
        Magnitude = Math.Sqrt(dx * dx + dy * dy);
        Direction = Math.Atan2(dy, dx);
    }
}

> ##### Create mutable state
>
> The preceding examples use primary constructor parameters to initialize readonly properties.
> You can also use primary constructors when the properties aren't readonly. Consider the following code:

In [ ]:
public struct Distance(double dx, double dy)
{
    public readonly double Magnitude => Math.Sqrt(dx * dx + dy * dy);
    public readonly double Direction => Math.Atan2(dy, dx);

    public void Translate(double deltaX, double deltaY)
    {
        dx += deltaX;
        dy += deltaY;
    }

    public Distance() : this(0,0) { }
}


>  The preceding struct is as though we had written the following code:

In [ ]:
public struct Distance
{
    private double __unspeakable_dx;
    private double __unspeakable_dy;

    public readonly double Magnitude => Math.Sqrt(__unspeakable_dx * __unspeakable_dx + __unspeakable_dy * __unspeakable_dy);
    public readonly double Direction => Math.Atan2(__unspeakable_dy, __unspeakable_dx);

    public void Translate(double deltaX, double deltaY)
    {
        __unspeakable_dx += deltaX;
        __unspeakable_dy += deltaY;
    }

    public Distance(double dx, double dy)
    {
        __unspeakable_dx = dx;
        __unspeakable_dy = dy;
    }
    public Distance() : this(0, 0) { }
}

> ##### Dependency injection
>
> Another common use for primary constructors is to specify parameters for dependency injection. 
>
> The following code creates a simple controller that requires a service interface for its use:

In [ ]:
public interface IService
{
    Distance GetDistance();
}

public class ExampleController(IService service) : ControllerBase
{
    [HttpGet]
    public ActionResult<Distance> Get()
    {
        return service.GetDistance();
    }
}

> ##### Initialize base class
>
> You can invoke a base class' primary constructor from the derived class' primary constructor.
> It's the easiest way for you to write a derived class that must invoke a primary constructor in the base class.

In [ ]:
public class BankAccount(string accountID, string owner)
{
    public string AccountID { get; } = accountID;
    public string Owner { get; } = owner;

    public override string ToString() => $"Account ID: {AccountID}, Owner: {Owner}";
}

In [ ]:
public class Account(string accountID, string owner)
{
    public string AccountID { get; } = ValidAccountNumber(accountID) 
        ? accountID 
        : throw new ArgumentException("Invalid account number", nameof(accountID));

    public string Owner { get; } = string.IsNullOrWhiteSpace(owner) 
        ? throw new ArgumentException("Owner name cannot be empty", nameof(owner)) 
        : owner;

    public override string ToString() => $"Account ID: {AccountID}, Owner: {Owner}";

    public static bool ValidAccountNumber(string accountID) => 
    accountID?.Length == 10 && accountID.All(c => char.IsDigit(c));
}

public class CheckingAccount(string accountID, string owner, decimal overdraftLimit = 0) : Account(accountID, owner)
{
    public decimal CurrentBalance { get; private set; } = 0;

    public void Deposit(decimal amount)
    {
        if (amount < 0)
        {
            throw new ArgumentOutOfRangeException(nameof(amount), "Deposit amount must be positive");
        }
        CurrentBalance += amount;
    }

    public void Withdrawal(decimal amount)
    {
        if (amount < 0)
        {
            throw new ArgumentOutOfRangeException(nameof(amount), "Withdrawal amount must be positive");
        }
        if (CurrentBalance - amount < -overdraftLimit)
        {
            throw new InvalidOperationException("Insufficient funds for withdrawal");
        }
        CurrentBalance -= amount;
    }
    
    public override string ToString() => $"Account ID: {AccountID}, Owner: {Owner}, Balance: {CurrentBalance}";
}

# Continue learning

There are plenty more resources out there to learn!

> [⏩ Next Module - Auto Property Initializer ](32.AutoPropertyInitializer.ipynb)
>
> [⏪ Last Module - Caller Information](../C#5.0/30.CallerInformation.ipynb)
>
> [Reference- Primary Constructors](https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/tutorials/primary-constructors)
>